In [1]:
%load_ext autoreload
%autoreload 2
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, diff_of_days, diff_of_times
from glob import glob
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
from train_prophet import main_prophet
import datetime
import math
import os
import numpy as np
import pandas as pd
import sys

HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/github/data_mining/library")
from parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C1')
               or path.count('C13')
               or path.count('V221')
               or path.count('V285')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('M')
               or path.count('Product')
              ]
test_paths  = [path for path in test_paths  
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C1')
               or path.count('C13')
               or path.count('V221')
               or path.count('V285')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('M')
               or path.count('Product')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)

Process ForkPoolWorker-108:
Process ForkPoolWorker-101:
Process ForkPoolWorker-97:
Process ForkPoolWorker-107:
Process ForkPoolWorker-13:
Process ForkPoolWorker-106:
Process ForkPoolWorker-92:
Process ForkPoolWorker-112:
Process ForkPoolWorker-35:
Process ForkPoolWorker-62:
Process ForkPoolWorker-14:
Process ForkPoolWorker-88:
Process ForkPoolWorker-55:
Process ForkPoolWorker-116:
Process ForkPoolWorker-28:
Process ForkPoolWorker-42:
Process ForkPoolWorker-78:
Process ForkPoolWorker-25:
Process ForkPoolWorker-64:
Process ForkPoolWorker-81:
Process ForkPoolWorker-43:
Process ForkPoolWorker-26:
Process ForkPoolWorker-36:
Process ForkPoolWorker-41:
Process ForkPoolWorker-111:
Process ForkPoolWorker-95:
Process ForkPoolWorker-17:
Process ForkPoolWorker-75:
Process ForkPoolWorker-37:
Process ForkPoolWorker-57:
Process ForkPoolWorker-32:
Process ForkPoolWorker-8:
Process ForkPoolWorker-15:
Process ForkPoolWorker-39:
Process ForkPoolWorker-1:
Process ForkPoolWorker-61:
Process ForkPoolWorker-

In [3]:
START_DATE = '2017-11-04'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
df_train['datetime'] = df_train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
df_test['datetime'] = df_test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
df_train['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
df_test['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
df_train['date'] = df_train['datetime'].map(lambda x: x.date())
df_test['date']  =  df_test['datetime'].map(lambda x: x.date())
df_train['hour'] = df_train['datetime'].map(lambda x: x.hour)
df_test['hour']  =  df_test['datetime'].map(lambda x: x.hour)

In [4]:
def trans_DT(df):
    list_regist = []
    for d, diff in tqdm(df[['date', 'D1']].values):
        if diff < 999999:
            regist = date_add_days(d, -1*diff)
        else:
            regist = date_add_days(d, 0)
        list_regist.append(str(regist))
    df['Regist_date'] = list_regist
    return df
df_train = trans_DT(df_train)
df_test  = trans_DT(df_test)

100%|██████████| 506691/506691 [00:01<00:00, 265139.40it/s]


In [5]:
df_train['is_train'] = 1
df_test['is_train']  = 0
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)

In [6]:
same_id_2 = [
2999837
,3056355
,3123212
,3133933
,3168396
,3207517
,3229603
,3246696
,3290427
,3325523
,3337729
,3355784
,3430130
,3441776
,3467658
,3511080
,3521314
,3542429
,3551764
,3671539
,3703201
,3733574
,3768559
]

# data[data[COLUMN_ID].isin(same_id_2)][sorted(list(set(check_cols)-set(['user_id'])))]
# data[data[COLUMN_ID].isin(same_id_3)][sorted(user_keys)]
# data['card5'].value_countse()

異なるuser keyで様々なユーザー特定パターン（正確でなくてもよい）を作り、それぞれのパターンで特徴を作る  
正確でなくてもよい理由としては、粒度を粗くしてもある条件を外した特徴量になるだけなので、問題ない

In [7]:
user_info = [
    'ProductCD',
#     'Regist_date'
]
list_card = [
    'card1',
    'card2',
    'card3',
    'card4',
#     'card5',
#     'card6',
]
list_addr = [
    'addr1',
    'addr2',
]
list_pemail = [
    'P_emaildomain'
]
list_M = [col for col in data.columns if col.startswith('M')]

user_keys__card = user_info + list_card

user_keys__card_addr = user_info + list_card + list_addr
user_keys__card_pemail = user_info + list_card + list_pemail
user_keys__card_M = user_info + list_card + list_M

user_keys__card_addr_pemail = user_info + list_card + list_addr + list_pemail
user_keys__card_addr_M = user_info + list_card + list_addr + list_M
user_keys__card_pemail_M = user_info + list_card + list_pemail + list_M

user_keys__card_addr_pemail_M = user_info + list_card + list_addr + list_pemail + list_M

list_user_keys = [
  ['user_keys__card',      user_keys__card,],
  ['user_keys__card_addr',      user_keys__card_addr,],
  ['user_keys__card_pemail',      user_keys__card_pemail,],
  ['user_keys__card_M',      user_keys__card_M,],
  ['user_keys__card_addr_M',      user_keys__card_addr_M,],
  ['user_keys__card_addr_pemail',      user_keys__card_addr_pemail,],
  ['user_keys__card_pemail_M',      user_keys__card_pemail_M,],
  ['user_keys__card_addr_pemail_M',      user_keys__card_addr_pemail_M,],
]
check_Ds = ['D1', 'D2', 'D3', 'D4', 'D5', 'datetime']

data['D1'].fillna(-1, inplace=True)
data[user_info + list_card] = data[user_info + list_card].astype('str').fillna('#')

# user_keys = user_keys__card_addr
# user_keys = user_keys__card_addr_pemail
user_keys = user_keys__card_addr_pemail_M

uid = 'user_id'
df = data.copy()

for col in user_keys:
    df[col].fillna('#', inplace=True)

df[user_keys] =  df[user_keys].astype('str')

In [8]:
print(user_keys)

['ProductCD', 'card1', 'card2', 'card3', 'card4', 'addr1', 'addr2', 'P_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']


user_keyで粗いグループを作ったら、そのグループの中で同じユーザーを探索し、条件が合えば仲間にしてさらにグループを作っていくイメージ

In [9]:
user_df = df.groupby(user_keys)[COLUMN_ID].min()
user_df = user_df.to_frame(uid)
user_df = df.set_index(user_keys).join(user_df)

check_cols = [COLUMN_ID, uid, COLUMN_TARGET, 'datetime', 'Regist_date', 'is_train'] + ['D1', 'D3', 'C1', 'C13', 'V221', 'V285']
user_df = user_df.reset_index()[check_cols]

user_df['C1']  += 1
user_df['C13'] += 1

if uid not in user_df.columns:
    user_df.reset_index(inplace=True)
    
user_df['hour'] = user_df['datetime'].map(lambda x: x.hour)
user_df['D3'].fillna(-99999, inplace=True)
user_df['C1'].fillna(-99999, inplace=True)
user_df['C13'].fillna(-99999, inplace=True)
user_df['V221'].fillna(-99999, inplace=True)
user_df['V285'].fillna(-99999, inplace=True)

train_max_datetime = df_train['datetime'].max()
test_min_datetime = df_train['datetime'].min()

# user_df.sort_values(by=[uid, 'datetime'], inplace=True)

# user_df['diff_from_regist'] = user_df['datetime'] - user_df['Regist_date'].map(lambda x: datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10]) ) )
# user_df['diff_from_regist'] = user_df['diff_from_regist'].map(lambda x: x.days)

# user_df['Regist_date_add_D1'] = user_df[['Regist_date', 'D1']].apply(lambda x: date_add_days(x[0], x[1]), axis=1)
# user_df['Regist_date_add_D1-D3'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]), axis=1)
# user_df['Regist_date_add_D1-D3-1'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]-1), axis=1)

# user_df['date__before1'] = user_df.groupby(uid)['date'].shift(1)
# user_df['date__before2'] = user_df.groupby(uid)['date'].shift(2)
# user_df['date__before3'] = user_df.groupby(uid)['date'].shift(3)
# user_df['D1__before1']   = user_df.groupby(uid)['D1'].shift(1)
# user_df['D1__before2']   = user_df.groupby(uid)['D1'].shift(2)

# user_df['diff_D1__before1'] = user_df[['D1', 'D1__before1']].apply(lambda x: x[0] - x[1] , axis=1)
# user_df['diff_D1__before2'] = user_df[['D1', 'D1__before2']].apply(lambda x: x[0] - x[1] , axis=1)
# user_df['diff_day__before1'] = user_df[['date', 'date__before1']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
# user_df['diff_day__before2'] = user_df[['date', 'date__before2']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
# user_df['diff_day__before3'] = user_df[['date', 'date__before3']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)

# user_df.to_csv('../output/0901_ieee__user_df_card_addr_pemail__for_specific_same_user.csv', index=False)

In [41]:
col_no = 'no'
user_df['no'] = np.arange(user_df.shape[0])+1

tmp = user_df.groupby(uid)['C1'].count().to_frame('user_cnt')
df_multi_user = tmp[tmp['user_cnt']>1]
multi_user = df_multi_user.index.tolist()
df_multi = user_df[user_df[uid].isin(multi_user)]

arg_list = get_parallel_arg_list(n_jobs=20, arg_list=multi_user)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(173762, 14)
(67858, 14)
(53214, 14)
(47794, 14)
(38480, 14)
(39696, 14)
(38105, 14)
(36768, 14)
(37981, 14)
(34371, 14)
(33979, 14)
(32225, 14)
(29680, 14)
(27671, 14)
(26699, 14)
(28230, 14)
(25621, 14)
(24383, 14)
(22975, 14)
(20133, 14)


In [51]:
#========================================================================
# テスト用
#========================================================================
# group_no_list = arg_list[0]
# for i in range(1):
def parallel_explore_match_user(df_base, group_no_list):
    
    list_user_map = []
    for group_no in group_no_list:

        group = df_base[df_base[uid]==group_no]
        group.sort_values(by='datetime', inplace=True)
        users = group[col_no].unique().tolist()
        user_map = {}
        cnt = 0
        
        while True:
        
            remain_user = group[~group[col_no].isin(list(user_map.keys()))][col_no].values
            if len(remain_user)==0:
                break
        #     remain_user = list( set(users) - set(list(user_map.keys())) )
        
            base_user = remain_user[0]
            user_map[base_user] = base_user
        
            base = group[group[col_no]==base_user]
            base_reg = base['Regist_date'].values[0]
            base_dt = base['datetime'].values[0]
            base_d1 = base['D1'].values[0]
            base_d3 = base['D3'].values[0]
            base_c1 = base['C1'].values[0]
            base_c13 = base['C13'].values[0]
            base_v221 = base['V221'].values[0]
            base_v285 = base['V285'].values[0]
            base_hour = base['hour'].values[0]
            base_fraud = base[COLUMN_TARGET].values[0]
            base_is_train = base['is_train'].values[0]
        
            for i in range(1, len(remain_user), 1):
        
                tmp_user = remain_user[i]
        
                tmp = group[group[col_no]==tmp_user]
                tmp_reg = tmp['Regist_date'].values[0]
                diff_regist_days = abs((pd.to_datetime(tmp_reg) - pd.to_datetime(base_reg)).days)
                
                # Regist_dateで1day以上の差は除外
                if diff_regist_days>1:
                    continue
                
                tmp_dt = tmp['datetime'].values[0]
                tmp_d1 = tmp['D1'].values[0]
                tmp_d3 = tmp['D3'].values[0]
                tmp_c1 = tmp['C1'].values[0]
                tmp_c13 = tmp['C13'].values[0]
                tmp_v221 = tmp['V221'].values[0]
                tmp_v285 = tmp['V285'].values[0]
                tmp_hour = tmp['hour'].values[0]
                tmp_fraud = tmp[COLUMN_TARGET].values[0]
                tmp_is_train = tmp['is_train'].values[0]
                
                is_same_data = base_is_train == tmp_is_train
                
                if not is_same_data:
                    if tmp_d3==-99999:
                        match_d1 = False
                    else:
                        no_term_date = date_add_days(pd.to_datetime(base_dt), tmp_d3*-1)
                        if train_max_datetime <= no_term_date <= test_min_datetime:
                            match_d1 = True
                        else:
                            match_d1 = False
                else:
                    match_d1 = False
        
                diff_d1   = tmp_d1 - base_d1
                diff_c1   = abs(tmp_c1 - base_c1)
                minus_c1   = tmp_c1 < base_c1
                if minus_c1:
                    thres_c1 = 3
                    thres_c1_2 = 3
                else:
                    thres_c1 = 3
                    thres_c1_2 = 6
                thres_c1_large = 5
                thres_v = 3
        
                diff_c13  = abs(tmp_c13 - base_c13)
                diff_v221 = abs(tmp_v221 - base_v221)
                diff_v285 = abs(tmp_v285 - base_v285)
        
                ratio_c1   = abs(tmp_c1  / base_c1)
                ratio_c13  = abs(tmp_c13 / base_c13)
        
        #         print("="*12)
        #         print(f"base {base_user} {base_dt}  ->  D1:{base_d1} D3:{base_d3} C1:{base_c1} C13:{base_c13} V221:{base_v221} V285:{base_v285}")
        #         print(f"tmp  {tmp_user} {tmp_dt}  ->  D1:{tmp_d1} D3:{tmp_d3} C1:{tmp_c1} C13:{tmp_c13} V221:{tmp_v221} V285:{tmp_v285}")
        #         print(f"diff_D1 {diff_d1} diffD1-D3: {diff_d1-tmp_d3} diff_C1:{diff_c1} diff_C13:{diff_c13} diff_V221:{diff_v221} diff_V285:{diff_v285}")
        #         print("="*12)
        
                # D3がNullの場合
                if base_d3==-99999 or tmp_d3==-99999:
                    # TODO
                    # ここでFraud条件を入れると、train-test間を同じユーザーにまとめることはできなくなる
                    # 両パターンを作成し、Fraud含めないパターンでまたいたユーザーが本当にまたいでるかチェックしたい
                    
                    if diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
                        pass
                    else:
                        continue
                    
        #             if base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
        #                 pa
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
        #                 pass
        #             else:
        #                 continue
        
                # D3がNullsでない場合
                # D3の差分が時差を考慮してもおかしい場合は別ユーザーにする（時差が1~2時間以上だと強制的にはじいてしまうが）
                elif (diff_d1 == tmp_d3) or (abs(diff_d1 - tmp_d3)==1):
                    pass
                else:
                    # train testの境目
                    if not is_same_data and match_d1:
                        pass
                    else:
                        # 別ユーザーとみなす
                        continue
        
                # D3がNullsでない場合
                if base_d3!=-99999 and tmp_d3!=-99999:
                    
                    # 別ユーザーとみなす
                    if diff_c1<-10 or diff_c1>1000:
                        continue
                    if diff_c13<-10 or diff_c13>1000:
                        continue
        
        #             print("  * check diff_V")
                    # V221, 285の変化量が1トランザクションで10以上はないという想定
                    if (10<= diff_v221) and (diff_v221<=100):
                        continue
                    if (10<= diff_v285) and (diff_v285<=100):
                        continue
        
        #             print("  * check diff_C")
                    if diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1>0:
        #                 print('    * pass1')
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1==0:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_d1!=0:
                        pass
                    else:
                        # 別ユーザーとみなす
                        continue
        
                user_map[tmp_user] = base_user
        #         print(pd.Series(user_map))
                
                # base_userと同じと判定されたので、baseを直近のトランザクションに書き換える
                base_dt    = tmp['datetime'].values[0]
                base_d1    = tmp['D1'].values[0]
                base_d3    = tmp['D3'].values[0]
                base_c1    = tmp['C1'].values[0]
                base_c13   = tmp['C13'].values[0]
                base_v221  = tmp['V221'].values[0]
                base_v285  = tmp['V285'].values[0]
                base_hour  = tmp['hour'].values[0]
                base_fraud = tmp[COLUMN_TARGET].values[0]
        
            if len(user_map.keys()) == len(users):
                break
#             break
                
        ser_user = pd.Series(user_map).to_frame('same_user')
        ser_user['group_no'] = group_no
        list_user_map.append(ser_user)
    return list_user_map

In [26]:
list_p = Parallel(20)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[12:], arg_list[12:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg1 = pd.concat(list_df, axis=0)

100%|██████████| 8/8 [00:15<00:00,  2.02s/it]


In [42]:
remain_users = []
for args in  arg_list[:12]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=20, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(134351, 14)
(55752, 14)
(39693, 14)
(34570, 14)
(30466, 14)
(29595, 14)
(26643, 14)
(23359, 14)
(22019, 14)
(24351, 14)
(23734, 14)
(21314, 14)
(22031, 14)
(23460, 14)
(22312, 14)
(21412, 14)
(20169, 14)
(20669, 14)
(19286, 14)
(19047, 14)


In [45]:
list_p = Parallel(20)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[4:], arg_list[4:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg2 = pd.concat(list_df, axis=0)

100%|██████████| 16/16 [00:19<00:00,  1.15s/it]


In [46]:
remain_users = []
for args in  arg_list[:4]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=20, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(63154, 14)
(18285, 14)
(18732, 14)
(19659, 14)
(14518, 14)
(11404, 14)
(12842, 14)
(11849, 14)
(10539, 14)
(9083, 14)
(9215, 14)
(8135, 14)
(8045, 14)
(6757, 14)
(7569, 14)
(6721, 14)
(7633, 14)
(6204, 14)
(7790, 14)
(6232, 14)


In [52]:
list_p = Parallel(20)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[5:], arg_list[5:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg3 = pd.concat(list_df, axis=0)

100%|██████████| 15/15 [00:03<00:00,  5.09it/s]


In [53]:
remain_users = []
for args in  arg_list[:5]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=16, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(30754, 14)
(27100, 14)
(4740, 14)
(3999, 14)
(4233, 14)
(8300, 14)
(6975, 14)
(4768, 14)
(4519, 14)
(5998, 14)
(6396, 14)
(7664, 14)
(5240, 14)
(3889, 14)
(4095, 14)
(5678, 14)


In [55]:
list_p = Parallel(16)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[2:], arg_list[2:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg4 = pd.concat(list_df, axis=0)

100%|██████████| 14/14 [00:01<00:00,  9.15it/s]


In [56]:
remain_users = []
for args in  arg_list[:2]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=12, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(1702, 14)
(10830, 14)
(2468, 14)
(1667, 14)
(10531, 14)
(3556, 14)
(5996, 14)
(4460, 14)
(5444, 14)
(1173, 14)
(7708, 14)
(2319, 14)


In [59]:
list_p = Parallel(10)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[5:], arg_list[5:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg5 = pd.concat(list_df, axis=0)

100%|██████████| 7/7 [00:00<00:00, 73.39it/s]


In [56]:
remain_users = []
for args in  arg_list[:5]:
    remain_users += args
# arg_list = get_parallel_arg_list(n_jobs=12, arg_list=remain_users)
# arg_df_list = []
# for user_ids in arg_list:
tmp_arg = user_df[user_df[uid].isin(remain_users)]
#     print(tmp.shape)
#     arg_df_list.append(tmp)

(1702, 14)
(10830, 14)
(2468, 14)
(1667, 14)
(10531, 14)
(3556, 14)
(5996, 14)
(4460, 14)
(5444, 14)
(1173, 14)
(7708, 14)
(2319, 14)


In [55]:
# list_p = Parallel(16)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[2:], arg_list[2:])])
tmp_list = parallel_explore_match_user(tmp_arg, remain_uers)
df_arg6 = pd.concat(tmp_list, axis=0)

100%|██████████| 14/14 [00:01<00:00,  9.15it/s]


In [ ]:
df_arg6

以下はテスト用

In [ ]:
#========================================================================
# テスト用
#========================================================================

tmp_check = user_df[user_df[COLUMN_ID].isin(same_id_3)]
check_user = tmp_check[uid].unique()
check = user_df[user_df[uid].isin(check_user)]
check.shape

# group = user_df[user_df[uid]==group_no]
group = check
group.sort_values(by='datetime', inplace=True)
users = group[col_no].unique().tolist()
user_map = {}

while True:

    remain_user = group[~group[col_no].isin(list(user_map.keys()))][col_no].values
#     remain_user = list( set(users) - set(list(user_map.keys())) )

    base_user = remain_user[0]
    user_map[base_user] = base_user
    list_same_user = []

    base = group[group[col_no]==base_user]
    base_reg = base['Regist_date'].values[0]
    base_dt = base['datetime'].values[0]
    base_d1 = base['D1'].values[0]
    base_d3 = base['D3'].values[0]
    base_c1 = base['C1'].values[0]
    base_c13 = base['C13'].values[0]
    base_v221 = base['V221'].values[0]
    base_v285 = base['V285'].values[0]
    base_hour = base['hour'].values[0]
    base_fraud = base[COLUMN_TARGET].values[0]
    base_is_train = base['is_train'].values[0]

    for i in range(1, len(remain_user), 1):

        tmp_user = remain_user[i]

        tmp = group[group[col_no]==tmp_user]
        tmp_reg = tmp['Regist_date'].values[0]
        diff_regist_days = abs((pd.to_datetime(tmp_reg) - pd.to_datetime(base_reg)).days)
        
        # Regist_dateで1day以上の差は除外
        if diff_regist_days>1:
            continue
        
        tmp_dt = tmp['datetime'].values[0]
        tmp_d1 = tmp['D1'].values[0]
        tmp_d3 = tmp['D3'].values[0]
        tmp_c1 = tmp['C1'].values[0]
        tmp_c13 = tmp['C13'].values[0]
        tmp_v221 = tmp['V221'].values[0]
        tmp_v285 = tmp['V285'].values[0]
        tmp_hour = tmp['hour'].values[0]
        tmp_fraud = tmp[COLUMN_TARGET].values[0]
        tmp_is_train = tmp['is_train'].values[0]
        
        is_same_data = base_is_train == tmp_is_train
        
        if not is_same_data:
            no_term_date = date_add_days(pd.to_datetime(base_dt), tmp_d3*-1)
            if train_max_datetime <= no_term_date <= test_min_datetime:
                match_d1 = True
        else:
            match_d1 = True

        diff_d1   = tmp_d1 - base_d1
        diff_c1   = abs(tmp_c1 - base_c1)
        minus_c1   = tmp_c1 < base_c1
        if minus_c1:
            thres_c1 = 3
            thres_c1_2 = 3
        else:
            thres_c1 = 4
            thres_c1_2 = 8
        thres_c1_large = 5
        thres_v = 3

        diff_c13  = abs(tmp_c13 - base_c13)
        diff_v221 = abs(tmp_v221 - base_v221)
        diff_v285 = abs(tmp_v285 - base_v285)

        ratio_c1   = abs(tmp_c1  / base_c1)
        ratio_c13  = abs(tmp_c13 / base_c13)

#         print("="*12)
#         print(f"base {base_user} {base_dt}  ->  D1:{base_d1} D3:{base_d3} C1:{base_c1} C13:{base_c13} V221:{base_v221} V285:{base_v285}")
#         print(f"tmp  {tmp_user} {tmp_dt}  ->  D1:{tmp_d1} D3:{tmp_d3} C1:{tmp_c1} C13:{tmp_c13} V221:{tmp_v221} V285:{tmp_v285}")
#         print(f"diff_D1 {diff_d1} diffD1-D3: {diff_d1-tmp_d3} diff_C1:{diff_c1} diff_C13:{diff_c13} diff_V221:{diff_v221} diff_V285:{diff_v285}")
#         print("="*12)

        # D3がNullの場合
        if base_d3==-99999 or tmp_d3==-99999:
            # TODO
            # ここでFraud条件を入れると、train-test間を同じユーザーにまとめることはできなくなる
            # 両パターンを作成し、Fraud含めないパターンでまたいたユーザーが本当にまたいでるかチェックしたい
            
            if diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
                pass
            else:
                continue
            
#             if base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
#                 pass
#             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
#                 pass
#             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
#                 pass
#             else:
#                 continue

        # D3がNullsでない場合
        # D3の差分が時差を考慮してもおかしい場合は別ユーザーにする（時差が1~2時間以上だと強制的にはじいてしまうが）
        elif (diff_d1 == tmp_d3) or (abs(diff_d1 - tmp_d3)==1):
            pass
        else:
            # train testの境目
            if not is_same_data and match_d1:
                pass
            else:
                # 別ユーザーとみなす
                continue

        # D3がNullsでない場合
        if base_d3!=-99999 and tmp_d3!=-99999:
            
            # 別ユーザーとみなす
            if diff_c1<-10 or diff_c1>1000:
                continue
            if diff_c13<-10 or diff_c13>1000:
                continue

#             print("  * check diff_V")
            # V221, 285の変化量が1トランザクションで10以上はないという想定
            if (10<= diff_v221) and (diff_v221<=100):
                continue
            if (10<= diff_v285) and (diff_v285<=100):
                continue

#             print("  * check diff_C")
            if diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1>0:
#                 print('    * pass1')
                pass
            elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1==0:
                pass
            elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.3<=ratio_c1 and ratio_c1<=1.6) and (0.3<=ratio_c13 or ratio_c13<=1.6) and diff_d1!=0:
                pass
            else:
                # 別ユーザーとみなす
                continue

        user_map[tmp_user] = base_user
#         print(pd.Series(user_map))
        
        # base_userと同じと判定されたので、baseを直近のトランザクションに書き換える
        base_dt    = tmp['datetime'].values[0]
        base_d1    = tmp['D1'].values[0]
        base_d3    = tmp['D3'].values[0]
        base_c1    = tmp['C1'].values[0]
        base_c13   = tmp['C13'].values[0]
        base_v221  = tmp['V221'].values[0]
        base_v285  = tmp['V285'].values[0]
        base_hour  = tmp['hour'].values[0]
        base_fraud = tmp[COLUMN_TARGET].values[0]

    if len(user_map.keys()) == len(users):
        break

tmp_user = pd.Series(user_map).to_frame('same_user')
check.set_index('no').join(tmp_user['same_user'])

In [376]:
pd.set_option('max_rows', 200)
# テスト用
# check_result = check.set_index('no').join(tmp_user['same_user'])
# check_result.sort_values(by=['same_user', 'datetime'], inplace=True)
# check_result.tail(804).head(200)
check_result[check_result['same_user']==4736]

,TransactionID,user_id,isFraud,datetime,Regist_date,is_train,D1,D3,C1,C13,V221,V285,hour,same_user
no,,,,,,,,,,,,,,
4736,3668748,2987205,NaN,2018-06-06 19:51:27,2018-01-05,0,152.0,153.0,105.0,8.0,3.0,0.0,19,4736
4763,3837824,2987205,NaN,2018-08-13 14:51:38,2018-01-05,0,220.0,68.0,89.0,5.0,0.0,0.0,14,4736


In [ ]:
1